In [1]:
import numpy as np
import matplotlib.pyplot as plt
import time

In [ ]:
## define functions
# compute confidence based on recency of memory/reward received
def make_pvals(p, envelope=15):
    return 1/np.cosh(p/envelope)

# retrieve relevant items from memory
def cosine_sim(key, dictionary, **kwargs):
    similarity_threshold = kwargs.get('threshold', 0.9)

    mem_cache = np.asarray(list(dictionary.keys()))
    entry     = np.asarray(key)

    mqt     = np.dot(mem_cache, entry)
    norm    = np.linalg.norm(mem_cache, axis = 1) * np.linalg.norm(entry) 

    cosine_similarity = mqt/norm

    index = np.argmax(cosine_similarity)
    similar_activity = mem_cache[index]
    if max(cosine_similarity) >= similarity_threshold:
        return similar_activity, index, max(cosine_similarity)

    else:
        #print('max memory similarity:', max(cosine_similarity))
        return [],[], max(cosine_similarity)
    
def make_cs(rwd, **kwargs):
    envelope = kwargs.get('envelope',10)
    tslr = []
    time_since_last_reward = np.nan 
    for i in rwd:
        tslr.append(time_since_last_reward)
        if i == 0:
            time_since_last_reward +=1
        if i == 1:
            time_since_last_reward = 0
            
    return make_pvals(np.asarray(tslr), envelope=envelope)


# make dummy list of rewards received
def make_spikelist(alpha):
    return np.random.choice([0,1], 100, p=[1-alpha,alpha])

# make dummy dictionary 
def make_dict(length, **kwargs):
    key_length = kwargs.get('key_length', 5)
    mem_length = kwargs.get('mem_length', length*10)
    d = {}
    for i in range(length):
        #make dict key
        k = tuple(np.random.choice(np.arange(10),key_length))
        num_actions = 6
        dataframe = np.empty((num_actions, 2))
        dataframe[:,0] = np.nan
        dataframe[:,1] = np.inf
        d[k] = [dataframe, -1]
        for j in range(num_actions -1):
            timestamp = np.random.randint(0, mem_length)
            deltaval = np.random.randn()
            
            d[k][0][j] = [deltaval, int(timestamp)]
            d[k][1]    = max(d[k][1], timestamp)
    return d


def softmax(x, T=1):
     """Compute softmax values for each sets of scores in x."""
     e_x = np.exp((x - np.max(x))/T)
     return e_x / e_x.sum(axis=0) # only difference
    

def plot_softmax(x, orig=False):
    for i in range(len(x)):
        y = softmax(x[i])
        if orig:
            f, axarr = plt.subplots(2, sharex=True)
            axarr[0].bar(np.arange(len(x[i])), x[i])
            axarr[1].bar(np.arange(len(x[i])), y) 
        else: 
            plt.bar(np.arange(len(x[i]))+0.1*i, y, width =0.1, alpha = 0.3)
    plt.show()

In [ ]:
d = make_dict(10)
d_items = np.asarray(list(d.items()))
print(d_items)
a = [x for y, x  in d_items[:,1]]
#print(a,a.index(min(a)), d_items[a.index(min(a)),0])

test_tuple = (1,1,1,1,1)
b = cosine_sim(test_tuple, d)
print(b)

In [ ]:
mem_, i, sim = cosine_sim(test_tuple, d, threshold = 0.5)
memory = d[tuple(mem_)]

confidence_score = sim*make_pvals(100 - memory[1], envelope=15)
print(confidence_score)

xx = np.asarray(memory)[0][:,0]
tt = np.asarray(memory)[0][:,1]
ab = np.nan_to_num(tt)
bc = 100-ab
print(tt,'\n', make_pvals(bc, envelope =50))
yy = xx.copy()



yy[5] = 0
zz = xx.copy()
zz[5] = np.average(zz[0:5])

print(yy)
print(zz)
ww = np.ones(len(xx))
plot_softmax([zz,yy, ww])




print(try_pvals(xx,15))

In [ ]:
## MFC confidence score tests

## Plot showing different confidence score decay envelopes
test = np.zeros(100)
test[0]= 1

plt.figure()
for i in range(10):
    env = i+ 0.001
    cs = make_cs(test,envelope=env)
    #print(i, np.where(cs<0.01)[0][0]) #how many steps before cs reaches zero
    plt.plot(cs,label=f'{np.round(env,1)}')
plt.legend(bbox_to_anchor = (1.25,.75))
plt.show()
plt.close()

### plot showing decay after randomly distributed rewards
reward_frequency = 0.1
cs_envelope = 1.5

sl = make_spikelist(reward_frequency)
sl1 = np.where(sl==1)[0]

cs = make_cs(sl,envelope=cs_envelope)

plt.figure()
plt.plot(sl1, np.ones_like(sl1) ,'o', label='Reward')
plt.plot(cs,label=f'{np.round(cs_envelope,1)}')
plt.legend(bbox_to_anchor = (1.25,.75))
plt.show()
plt.close()

In [ ]:
## Test dictionary for memory confidence tests
## show confidence in a dictionary entry
key_length = 10
dic_length = 50
num_sample = 1000

d = make_dict(dic_length, key_length=key_length)
pn = {}
js = [1.5/cs_envelope]
for j in js:#range(1,11):
    pn[str(j)] = [[],[],[]]
    #d = make_dict(dic_length, key_length=key_length)
    for i in range(num_sample): 
        test_key = tuple(np.random.choice(np.arange(10), key_length))
        sim_key, index = cosine_sim(test_key, d)

        if len(sim_key) is not 0:
            envelope = cs_envelope*j*10
            timestamp, deltavalue = d[tuple(sim_key)]
            confidence = make_pvals(timestamp, envelope)
            delta_scaled_conf = deltavalue*make_pvals(timestamp, envelope)
            pn[str(j)][0].append(timestamp)
            pn[str(j)][1].append(confidence)
            pn[str(j)][2].append(delta_scaled_conf)
        else:
            pass

In [ ]:
fig, ax = plt.subplots(2,1,sharex=True)

for j in js:
    label = f'{cs_envelope*j*10}'

    #ax[0].scatter(pn[str(j)][0], pn[str(j)][1], label=label)
    #ax[1].scatter(pn[str(j)][0], pn[str(j)][2], label=label)
    
    # plot time-dependent confidence
    test = zip(pn[str(j)][0], pn[str(j)][1])
    sort_test = sorted(test, key = lambda x: x[0])
    ax[0].plot(np.array(sort_test)[:,0], np.array(sort_test)[:,1], label=label)
    
    # plot delta-scaled confidence
    test = zip(pn[str(j)][0], pn[str(j)][2])
    sort_test = sorted(test, key = lambda x: x[0])
    ax[1].plot(np.array(sort_test)[:,0], np.array(sort_test)[:,1], label=label)
    
    
ax[0].legend(loc='upper center', bbox_to_anchor=(.5,1.3), ncol =3)
plt.show()
plt.close()

In [ ]:
a = np.empty((6,2))
a[:,0] = np.nan
a[:,1] = 0 
a[2] = [10, 2]
a[4] = [1, -4]
print(a)

def sigmoid(x):
    return 1 / (1 + np.exp(-x))
x = make_pvals(a[:,0], 15)

print(np.arange(len(x)))

plt.figure()
plt.scatter(np.arange(len(x)), x)
plt.scatter(np.arange(len(x)), np.multiply(x, sigmoid(a[:,1])))

In [ ]:
# need to create arbitrator to decide between confidence of MFC and EC
''' 
option 1: softmax MF_cs and EC_cs 
    use this as a probability distribution to select either MF or EC

# pros:
 don't have to worry about convex combinations in nonconvex spaces 
# cons:
    20% certainty is not good, you want to heavily favour 80% confidence? 

option 2: combine pi_mf and pi_ec weighted by their respective cs scores
# pros:

# cons: 
'''


In [ ]:
## test how long a cosine similarity calculation takes for different key lengths
num_runs = 40
key_length = 100 # len(list(d.keys())[0])
d = make_dict(50, key_length = key_length)
avg_h = 0
for i in range(num_runs):
    test_key = tuple(np.random.choice(np.arange(10), key_length))
    start_time = time.time()
    cosine_sim(test_key, d)
    avg_h += time.time()-start_time
print(f'average run time is {avg_h/num_runs}')